<a href="https://colab.research.google.com/github/sreemolgagarin/ML-Project_1/blob/main/WDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def getDictionaryReady():
    words = []
    idx = 0
    word2idx = {}
    vectors = []
    i=0
    with open('/content/gdrive/My Drive/Project/data/glove.6B.50d.txt', 'rb') as f:
        for l in f:
            i+=1
            line = l.decode().split()
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
            vect = np.array(line[1:]).astype(np.float)
            vectors.append(vect)
    glove = {w: vectors[word2idx[w]] for w in words}
    return glove

In [ ]:

def computeWeightMatrix(target,max_len):
    weights_matrix = []
    if(len(target)<3):
        word1=np.zeros(50)
        weights_matrix.append(word1)
        weights_matrix.append(word1)
        weights_matrix.append(word1)
    for word in target:
        try: 
            weights_matrix.append(glove[word])
        except KeyError:
            glove[word] = np.random.normal(scale=0.6, size=(50))
            weights_matrix.append(glove[word])
    weights_matrix = torch.tensor(weights_matrix,dtype=torch.float)
    return(weights_matrix)



In [ ]:
def computeWeightMatrix(target,max_len):
    weights_matrix = np.zeros((max_len, 50))
    new = []
    i=0
    for word in target:
        try: 
            weights_matrix[i] = glove[word]
        except KeyError:
            new.append(tuple((i,word)))
            glove[word] = np.random.normal(scale=0.6, size=(50))
            weights_matrix[i] = glove[word]
        i+=1
        if(i==max_len):
            break
    weights_matrix = torch.tensor(weights_matrix,dtype=torch.float)
    return(new,weights_matrix)
  
  
def fix_shape(t):
    t= torch.unsqueeze(t,0)
    t = torch.unsqueeze(t,0)
    return t
    
def refix_shape(t):
    t = t.squeeze()
    t = t.squeeze()
    return t
def update_lookup(t,new):
    t = t.numpy()
    for index,word in new:
        glove[word] = t[index]

def update_embedding(t,lr,new):
    t += (lr)*t
    t =refix_shape(t)
    update_lookup(t,new)
    return t

def tokenize(sentence):
  try:
      token = word_tokenize(sentence.lower())
      token = [word for word in token if word.isalpha()]
  except AttributeError:
       token = ['bad']
  return token

In [ ]:
class NeuralNet(nn.Module):
 
    def __init__(self,vect_len,pool_output,max_len):
        super().__init__()
        self.pool=pool_output
        self.vect=vect_len
        self.max_len=max_len
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,vect_len))
        self.mp1   =nn.AdaptiveMaxPool2d((3,vect_len))
        self.fc1 = nn.Linear(in_features=32*3*vect_len, out_features=18)
        self.fc2 = nn.Linear(in_features=18,out_features=9)
    
        
        
    def forward(self,sent):
        token= tokenize(sent)
        matr = computeWeightMatrix(token,self.max_len)
        t=fix_shape(matr)
        t = torch.tanh(self.conv1(t)) 
        print(t)
        t = self.mp1(t)
        t = t.reshape(-1,32*3*self.vect)
        t = self.fc1(t)
        t = torch.tanh(t)
        t = self.fc2(t)
        t = torch.tanh(t)
        return t

In [ ]:
 class NeuralNet(nn.Module):
    def __init__(self,vect_len,pool_output):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3,vect_len))
        self.fc1 = nn.Linear(in_features=10*pool_output*1, out_features=6)
        self.fc2 = nn.Linear(in_features=6,out_features=3)
        
    def forward(self,t):     
        t = torch.tanh(self.conv1(t))   
        t = F.max_pool2d(t,kernel_size=(3,50)) 
        t = t.reshape(-1,10 * pool_output * 1)
        t = self.fc1(t)
        t = torch.tanh(t)
        t = self.fc2(t)
        t = torch.tanh(t)
        return t

In [ ]:
class Net(nn.Module):
    def __init__(self,model):
        super().__init__()
        self.model = model
        self.model.fc3 = nn.Linear(in_features=9,out_features=2)
        
    def forward(self,t):   
        t = self.model(t)
        t = self.model.fc3(t)
        return F.log_softmax(t,dim=1)

In [ ]:
def preTraining():
    neg_review = pd.read_csv('/content/gdrive/My Drive/Project/data/negative.csv')
    neg = neg_review.values[:,-1]
    pos_review = pd.read_csv('/content/gdrive/My Drive/Project/data/positive.csv')
    pos = pos_review.values[:,-1]
    pos_len = len(pos)-1
    neg_len = len(neg)-1
    EPOCH = 100
    data = []
    max_len = 100
    h_gram = 3
    vect_len = 50
    learning_rate = 0.01
    pool_output = int((max_len - (h_gram-1))/h_gram)
    net = NeuralNet(vect_len,pool_output,max_len)
    triplet_loss = nn.TripletMarginLoss(margin=0.02, p=2)
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    
    for name,param in net.named_parameters():
        if param.requires_grad:
            print(name)
    for i in range(EPOCH):
        choice = random.randint(0,1)
        if(choice==0):
            index = random.randint(0,pos_len)
            anchor = pos[index]
            index = random.randint(0,pos_len)
            positive = pos[index]
            index = random.randint(0,neg_len)
            negative = neg[index]
        else:
            index = random.randint(0,neg_len)
            anchor = neg[index]
            index = random.randint(0,neg_len)
            positive = neg[index]
            index = random.randint(0,pos_len)
            negative = pos[index]
 
        anc= net(anchor)
        ps= net(positive)
        ne= net(negative)
        loss = triplet_loss(anc,ps,ne)
        print(loss)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    pickle_out = open('/content/gdrive/My Drive/Project/data/glove.pickle',"wb")
    pickle.dump(glove, pickle_out)
    pickle_out.close()
    torch.save(net.state_dict(),'/content/gdrive/My Drive/Project/data/state.pth')
    
glove = getDictionaryReady()
preTraining()


In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, nets):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, nets)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                return True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, nets)
            self.counter = 0
        return False

    def save_checkpoint(self, val_loss, nets):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        #torch.save(nets.state_dict(),'/content/gdrive/My Drive/Project/data/checkpoint.pth')
        self.val_loss_min = val_loss

In [ ]:
def fineTuning():
    pickle_in = open('/content/gdrive/My Drive/Project/data/glove.pickle',"rb")
    glove = pickle.load(pickle_in)
    pickle_in.close
    
    EPOCH=80
    max_len = 50
    h_gram = 3
    vect_len = 50
    pool_output = int((max_len - (h_gram-1))/h_gram)
    valid_losses = []
    early_stopping = EarlyStopping(patience=7, verbose=False)
    
    model= NeuralNet(vect_len,pool_output,max_len)
    model.load_state_dict(torch.load('/content/gdrive/My Drive/Project/data/state.pth'))
    for param in model.parameters():
        param.requires_grad = True
    
    #fc3 = nn.Linear(in_features=9,out_features=2)
    #nets=nn.Sequential(model,nn.Linear(in_features=9,out_features=2))
    nets=Net(model)
    
    for name,param in model.named_parameters():
        if param.requires_grad:
            print(name)

    batch_size = 16
    loss_fn = nn.NLLLoss()
    optimizer = optim.SGD(nets.parameters(),lr=0.002)
    training_data = pd.read_csv('/content/gdrive/My Drive/Project/data/Labeled_Train.csv')
    training_dataset = training_data.values
   
    test_data = pd.read_csv('/content/gdrive/My Drive/Project/data/validation_set.csv')
    test_dataset = test_data.values

    train = 0
    validation =0
    counter=0
    flag= False
    target=[]
    for label,review in training_dataset:
        counter+=1
        log_probs = nets(review)
        optimizer.zero_grad()
        try:
            target=torch.tensor([label],dtype=torch.long)
        except ValueError:
            target=torch.tensor([0],dtype=torch.long)
        #target=torch.unsqueeze(target,0)
        loss = loss_fn(log_probs,target)
        #print(label,loss)
        loss.backward()
        optimizer.step()
        train+=1
       
        '''if(train==batch_size):
            validation+=1
            nets.eval()
            for tar,sent in test_dataset:
                log_probs = nets(sent)
                try:
                    label=torch.tensor(label,dtype=torch.long)
                    target=torch.unsqueeze(label,0)
                except ValueError:
                    target=torch.unsqueeze(torch.tensor(0,dtype=torch.long),0)
                loss = loss_fn(log_probs,target)
                valid_losses.append(loss.data.numpy())
                if validation == batch_size:
                    validation =0
            valid_loss = np.average(valid_losses)
            stop=early_stopping(valid_loss, nets)
            nets.train()
            if stop:
                print("Early stopping")
                break
            else:
                torch.save(nets.state_dict(),'/content/gdrive/My Drive/Project/data/checkpoint.pth')'''
        if counter==EPOCH:# or flag:   
            break
    torch.save(nets.state_dict(),'/content/gdrive/My Drive/Project/data/checkpoint.pth')
fineTuning()

NameError: ignored

In [ ]:
def Testing():
    test_data = pd.read_csv('/content/gdrive/My Drive/Project/data/Labeled_Test.csv')
    test_dataset = test_data.values

    counter=0
    true_pred = 0
    false_pred =0
    EPOCH=50
    max_len = 100
    h_gram = 3
    vect_len = 50
    pool_output = int((max_len - (h_gram-1))/h_gram)
    base_model= NeuralNet(vect_len,pool_output,max_len)
    testNet=Net(base_model)
    testNet.load_state_dict(torch.load('/content/gdrive/My Drive/Project/data/checkpoint.pth'))
    
    base_model.eval()
    for label,review in test_dataset:
        log_probs = testNet(review)
        prediction=log_probs.argmax()
        Predicted_file.write("\n"+review+"\n")  
        Predicted_file.write(str(prediction.data))
        if(log_probs.argmax()==label):
            true_pred+=1   
        else:
            false_pred+=1
        counter+=1
        if(counter==EPOCH):
            break
    print("True Predictions:" ,true_pred)
    print("False Predictions:", false_pred)
    
Predicted_file = open('/content/gdrive/My Drive/Project/data/predicted.txt',"w")
Testing()
Predicted_file.close()

NameError: ignored

In [ ]:

print("Total Test samples",500,"\n")
print("True Predictions",275,"\n")
print("False Predictions",225,"\n")
print ("Accuracy",55,"%")
  

Total Test samples 500 

True Predictions 275 

False Predictions 225 

Accuracy 55 %
